In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error as MSE 

In [2]:
#! pip install lightgbm

DataDictionaryWiDS2021.csv    TrainingWiDS2021.csv
SampleSubmissionWiDS2021.csv  UnlabeledWiDS2021.csv
SolutionTemplateWiDS2021.csv  initial_research.ipynb


In [3]:
data = pd.read_csv('TrainingWiDS2021.csv',index_col=0)
data.head()

,encounter_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,...,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus
1,214826,118,68.0,22.732803,0,Caucasian,M,180.3,Floor,Floor,...,NaN,NaN,0,0,0,0,0,0,0,1
2,246060,81,77.0,27.421875,0,Caucasian,F,160.0,Floor,Floor,...,51.0,51.0,0,0,0,0,0,0,0,1
3,276985,118,25.0,31.952749,0,Caucasian,F,172.7,Emergency Department,Accident & Emergency,...,NaN,NaN,0,0,0,0,0,0,0,0
4,262220,118,81.0,22.635548,1,Caucasian,F,165.1,Operating Room,Operating Room / Recovery,...,337.0,337.0,0,0,0,0,0,0,0,0
5,201746,33,19.0,NaN,0,Caucasian,M,188.0,NaN,Accident & Emergency,...,NaN,NaN,0,0,0,0,0,0,0,0


In [4]:
cat = {'ethnicity':'category',
       'gender':'category', 
       'hospital_admit_source':'category',
       'icu_admit_source':'category',
       'icu_stay_type':'category', 
       'icu_type':'category'}

data = data.astype(cat)

In [5]:
X = data.drop('diabetes_mellitus', axis=1)
y = data.diabetes_mellitus

In [6]:
# Splitting 
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                      test_size = 0.3, random_state = 123) 

In [7]:
#!pip3 install wandb
#!wandb login

In [8]:
import wandb
from wandb.lightgbm import wandb_callback

wandb.init(project="wids_2021", sync_tensorboard=True)
config = wandb.config

wandb: Currently logged in as: mariapaskevich (use `wandb login --relogin` to force relogin)


In [9]:
params = {'learning_rate':0.05,
          'num_leaves':50,
          'n_estimators':1000,
          'metric': 'auc',
          'objective': 'binary'}


watchlist = [(X_train, 'train'), (X_test, 'test')]
num_round = 5

# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [10]:
# train

wandb.config.update(params)

# add lightgbm callback
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                valid_names=('auc'),
                callbacks=[wandb_callback()],
                early_stopping_rounds=50)

# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
# eval
#print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Info] Number of positive: 19698, number of negative: 71411
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.036258 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 26083
[LightGBM] [Info] Number of data points in the train set: 91109, number of used features: 178


/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.216203 -> initscore=-1.287935
[LightGBM] [Info] Start training from score -1.287935
[1]	auc's auc: 0.805103
Training until validation scores don't improve for 50 rounds
[2]	auc's auc: 0.809101
[3]	auc's auc: 0.813536
[4]	auc's auc: 0.814745
[5]	auc's auc: 0.816169
[6]	auc's auc: 0.818942
[7]	auc's auc: 0.819796
[8]	auc's auc: 0.821054
[9]	auc's auc: 0.821797
[10]	auc's auc: 0.822059
[11]	auc's auc: 0.822831
[12]	auc's auc: 0.823489
[13]	auc's auc: 0.823868
[14]	auc's auc: 0.82485
[15]	auc's auc: 0.826092
[16]	auc's auc: 0.826728
[17]	auc's auc: 0.827774
[18]	auc's auc: 0.828377
[19]	auc's auc: 0.828897
[20]	auc's auc: 0.82988
[21]	auc's auc: 0.830551
[22]	auc's auc: 0.831301
[23]	auc's auc: 0.831706
[24]	auc's auc: 0.832106
[25]	auc's auc: 0.83244
[26]	auc's auc: 0.83296
[27]	auc's auc: 0.833303
[28]	auc's auc: 0.833802
[29]	auc's auc: 0.834258
[30]	auc's auc: 0.834721
[31]	auc's auc: 0.835247
[32]	auc's auc: 0.835537
[33]	auc's auc: 0.